# ChemEnv : 自动识别坐标环境

识别坐标环境的步骤(coordination environments):    
1. 通过维诺分析(Voronoi), 在距离-角度网格上寻找邻位(neighbors); 
2. 对于模型环境中的所有距离和角度参数, 进行连续对称测量(CSMs, continous symmetry measures);    
3. 使用不同的策略对结果进行精修, [Statistical Analysis of Coordination Environments in Oxides](https://pubs.acs.org/doi/10.1021/acs.chemmater.7b02766);

分析少量结构, `get_environment`:    
`SimplestChemenvStrategy(distance_cutoff=1.4, angle_cutoff=0.3, Only anion-cation bonds continuous_symmetry_measure_cutoff=10.0)` visualize? 

分析坐标环境对于距离和角度参数的依赖, "See list of environments determined for each (unequivalent) site ?"` and choose a site.

In [1]:
import logging
from pymatgen.ext.matproj import MPRester
from pymatgen.analysis.chemenv.coordination_environments.coordination_geometry_finder import LocalGeometryFinder
from pymatgen.analysis.chemenv.coordination_environments.chemenv_strategies import SimplestChemenvStrategy, MultiWeightsChemenvStrategy
from pymatgen.analysis.chemenv.coordination_environments.structure_environments import LightStructureEnvironments

从MP获得alpha-石英的结构

In [2]:
a = MPRester()
struct = a.get_structure_by_material_id('mp-7000')

> alpha-quartz: <img src="https://github.com/materialsvirtuallab/matgenb/blob/master/notebooks/Figure_quartz.png?raw=true" width="30%">;    

In [3]:
%%pass #从cif文件导入结构
from pymatgen.io.cif import CifParser
parser = CifParser("mystructure.cif")
struct = parser.get_structures()[0]

## Setup of the local geometry finder (Voronoi analysis and csm on a grid follow)


Voronoi analysis: centering_types.   
Here, the structures are centered at the centroid (`centering_type='centroid'`) and the central site, for which the Voronoi analysis is performed, is included in the calculation of the centroid (`include_central_site_in_centroid=True`). See [the code](http://pymatgen.org/_modules/pymatgen/analysis/chemenv/coordination_environments/coordination_geometry_finder.html#LocalGeometryFinder.setup_parameters) for more information. Moreover, a logging is introduced. This is especially important if you run very large calculations (large distances and small angle parameters). 

设置(Setup)局部结构查找器

In [4]:
lgf = LocalGeometryFinder()
lgf.setup_parameters(centering_type='centroid', include_central_site_in_centroid=True)


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766



将日志存于文件

In [8]:
logging.basicConfig(filename='data2/chemenv_structure_environments.log',
                    format='%(levelname)s:%(module)s:%(funcName)s:%(message)s',
                    level=logging.DEBUG)
lgf.setup_structure(structure=struct)

## 获取结构环境

`maximum_distance_factor`: to save some computing time.Multiplied with the distance to the closest neighbour it results in the maximum distance that is considered in the Voronoi approach.   

Similarily, one could define a `minimum_angle_factor`. This factor is multiplied with the maximum solid angle to the neighbours to arrive at the minimum solid angle that is considered in the Voronoi analysis.

Moreover, one can also exclude atoms from the computation of the coordination environments (e.g., `se = lgf.compute_structure_environments(excluded_atoms=['O'])`). Additional parameters such as `only_atoms`, `only_indices` and `only_cations` in combination with `valences` might be helpful as well. 

In [9]:
se = lgf.compute_structure_environments(maximum_distance_factor=1.41,only_cations=False)

## Different strategies to analyze the StructureEnvironments

Now, the strategy to interpret the data from before is chosen to arrive at information about the coordination environments.
One can choose between two different types of strategies: we start with the `SimplestChemenvStrategy`. This strategy type uses fixed angle and distance parameters for the definition of neighbors in the Voronoi approach. 
The resulting coordination environment is uniquely defined and is then given as the one with the lowest continuous symmetry measure. One of the disadvantages is that it fails for intermediate coordination environments and depends very much on the cutoff parameters chosen. However, it is right in about 85% of the cases. 


The important parameters for this strategy are `distance_cutoff` and `angle_cutoff`. The neighbouring atoms that are considered in this approach have a **maximal distance** of `distance_cutoff` \* **the distance to the closest neighbour** and a **minimal solid angle** of `angle_cutoff` \* **the biggest solid angle**. For more information, see: [D. Waroquiers et al., *Chem Mater.*, **2017**, *29*, 8346](http://pubs.acs.org/doi/10.1021/acs.chemmater.7b02766) and [the code](http://pymatgen.org/_modules/pymatgen/analysis/chemenv/coordination_environments/chemenv_strategies.html#SimplestChemenvStrategy).

In [7]:
strategy = SimplestChemenvStrategy(distance_cutoff=1.4, angle_cutoff=0.3)
lse = LightStructureEnvironments.from_structure_environments(strategy=strategy, structure_environments=se)

Next, one can print the information on the coordination environments for each site in the structure. Here, the information for a site occupied by one oxygen is printed.

In [8]:
#print coordination environments for a special site
isite = 5
print(lse.coordination_environments[isite])

[{'ce_symbol': 'A:2', 'ce_fraction': 1.0, 'csm': 2.2602837757388414, 'permutation': [0, 1]}]


 `ce_symbol` symbolizes the coordination environment. `A:2` is the angular coordination environment. The full list of coordination environments can be seen in the supporting information of [D. Waroquiers et al., *Chem Mater.*, **2017**, *29*, 8346](http://pubs.acs.org/doi/10.1021/acs.chemmater.7b02766): [Supporting Information](http://pubs.acs.org/doi/suppl/10.1021/acs.chemmater.7b02766/suppl_file/cm7b02766_si_004.pdf). With this strategy, the `ce_fraction` is always equal to 1.0. The `csm` is the continous symmetry measure. This value lies between 0.0 and 100.0. At 0.0 the chemical environment in the structure is identical to the model environment and can be interpreted as a distance to a shape. In this example, a `csm` of 2.26 shows that the environment still shows some similarity to the model environment. Coordination environments with an `csm` greater than 2.5 are already considered as rather distorted. For more information on the CSM, have a look at the original publiciation: [M. Pinksky, D. Avnir, *Inorg. Chem.*, **1998**, *37*, 5575](http://pubs.acs.org/doi/abs/10.1021/ic9804925).

A more evolved strategy type, especially for intermediate coordination enviroments, and the one used in the publication [D. Waroquiers et al., *Chem Mater.*, **2017**, *29*, 8346](http://pubs.acs.org/doi/10.1021/acs.chemmater.7b02766) is the `MultiWeightsChemenvStrategy`. In the following, the parameters (weigths) from the publication are used. Of course, experts could also modify this. Have a close look at the code for this [here](http://pymatgen.org/_modules/pymatgen/analysis/chemenv/coordination_environments/chemenv_strategies.html#MultiWeightsChemenvStrategy). 

In [9]:
#Get the strategy from D. Waroquiers et al., Chem Mater., 2017, 29, 8346.
from pymatgen.analysis.chemenv.coordination_environments.chemenv_strategies import MultiWeightsChemenvStrategy
strategy = MultiWeightsChemenvStrategy.stats_article_weights_parameters()

lse = LightStructureEnvironments.from_structure_environments(strategy=strategy, structure_environments=se)

We start with the same oxygen site.

In [10]:
#print coordination environments for a special site
isite = 5
print(lse.coordination_environments[isite])

[{'ce_symbol': 'A:2', 'ce_fraction': 0.6735414322036577, 'csm': 2.2602837757388414, 'permutation': [0, 1]}, {'ce_symbol': 'L:2', 'ce_fraction': 0.3264585677963424, 'csm': 2.8902708777516026, 'permutation': [0, 1]}]


This time, two coordination environments for one site exist as indicated by two appearances of `ce_symbol` and there is also a `ce_fraction` for each of the environments. The latter indicates the coordination environment is an intermediate between `A:2` (angular) and `L:2` (linear) with 67% angular environment and 33% linear environment. Also, there is a `csm` value for both environments. As already indicated by the `ce_fraction`, the `csm` for `A:2` is lower (=in better agreement with the model environment) than the `csm` for `L:2`.

Now, an example follows where only one coordination environment exists. A Si occupies the corresponding site. 

In [11]:
#another site where you have only one coordination environment (tetrahedron, T:4)
isite = 1
print(lse.coordination_environments[isite])

[{'ce_symbol': 'T:4', 'ce_fraction': 1.0, 'csm': 0.009887786111312944, 'permutation': [0, 1, 2, 3]}]


The resulting coodination environment is `T:4` (tetrahedron). 